<a href="https://colab.research.google.com/github/allyan/NL2SQL/blob/main/Copy_of_bert_weaviate_information_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install transformers
!pip3 install nltk
!pip3 install torch
!pip3 install weaviate-client==3.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 15.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 KB 4.4 MB/s eta 0:00:00


## Import the BERT transformer model and pytorch

We are using the `bert-base-uncased` model in this example, but any model will work. Feel free to adjust accordingly.

## Initialize Weaviate Client
This assumes you have Weaviate running locally on `:8080`. Adjust URL accordingly. You could also enter the WCS URL here, for example, if you are running a WCS cloud instance instead of running Weaviate locally.

In [71]:
import torch
from transformers import AutoModel, AutoTokenizer
from nltk.tokenize import sent_tokenize
import weaviate

torch.set_grad_enabled(False)

# udpated to use different model if desired
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
model = AutoModel.from_pretrained(MODEL_NAME)
# model.to('cuda') # remove if working without GPUs
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# initialize nltk (for tokenizing sentences)
import nltk
nltk.download('punkt')

# initialize weaviate client for importing and searching
client = weaviate.Client("http://64.71.146.93:8080")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Load dataset from disk
Create some helper functions to create the dataset (20-newsgroup text posts) from disk. These methods are specific to the structure of your dataset, adjust accordingly.

In [72]:
import os
import random

questions = []
sqls = []

def read_dataset():
    file_data = open("sample_data/dataset.txt", 'r', encoding="utf8")
    for line in file_data:
        if line.split(" : ")[0] not in questions and line.split(" : ")[1] not in sqls:
            questions.append(line.split(" : ")[0])
            sqls.append(line.split(" : ")[1])

read_dataset()
print(len(questions))
       


429


## Vectorize Dataset using BERT

The following is a helper function to vectorize all posts (using our BERT transformer) which are entered as an array. The return array contains all the vectors in the same order. BERT is optimized to run on GPUs, if you're using CPUs this might take a while. 

In [73]:
import time

def text2vec(text):
    tokens_pt = tokenizer(text, padding=True, truncation=True, max_length=500, add_special_tokens = True, return_tensors="pt")
    outputs = model(**tokens_pt)
    # tokens_pt.to('cuda') # remove if working without GPUs
    return outputs[0].mean(0).mean(0).detach()

def vectorize_questions(posts=[]):
    post_vectors=[]
    before=time.time()
    for i, post in enumerate(posts):
        vec=text2vec(sent_tokenize(post))
        post_vectors += [vec]
        if i % 500 == 0 and i != 0:
            print("So far {} objects vectorized in {}s".format(i, time.time()-before))
    after=time.time()
    
    print("Vectorized {} items in {}s".format(len(posts), after-before))
    
    return post_vectors

### Run everything we have so far

It is now time to run the functions we defined before. Let's load 50 random posts from disk, then vectorize them using BERT.

## Initialize Weaviate

Now that we have vectors we can import both the posts and the vectors into Weaviate, so we can then search through them.

### Init a simple schema
Our schema is very simple, we just have one object class, the "Post". A post class has just a single property, which we call "content" and is of type "text".

Each class in schema creates one index, so by running the below we tell weaviate to create one brand new vector index waiting for us to import data.

In [76]:
# import json
schema = client.schema.get()
print(json.dumps(schema, indent=4))
class_obj = {
"class": "SQL_Questions", # <= Change to your class name - it will be your collection
"description": "SQL to Text mapping",
"vectorizer": "none",
"properties": [
    {
        "dataType": [
            "string"
        ],
        "description": "question",
        "name": "question"
    },
    {
        "dataType": [
            "string"
        ],
        "description": "sql",
        "name": "sql"
    }
]
}
client.schema.delete_class("SQL_Questions")
client.schema.create_class(class_obj)

{
    "classes": [
        {
            "class": "Weak_Sim_Intent",
            "description": "Add weak intent similarity",
            "invertedIndexConfig": {
                "bm25": {
                    "b": 0.75,
                    "k1": 1.2
                },
                "cleanupIntervalSeconds": 60,
                "stopwords": {
                    "additions": null,
                    "preset": "en",
                    "removals": null
                }
            },
            "properties": [
                {
                    "dataType": [
                        "int"
                    ],
                    "description": "Frame_nmb",
                    "name": "frame_nmbr"
                },
                {
                    "dataType": [
                        "string"
                    ],
                    "description": "movie_id",
                    "name": "movie_id",
                    "tokenization": "word"
                },
           

In [77]:

def import_questions_with_vectors(questions, sqls, vectors, client):
    if len(questions) != len(vectors):
        raise Exception("len of posts ({}) and vectors ({}) does not match".format(len(questions), len(vectors)))
        
    for i, question in enumerate(questions):
        try:
           client.data_object.create(
                data_object={"question": questions[i], "sql": sqls[i]},
                class_name='SQLQuestions',
                vector=vectors[i]
            )
        except Exception as e:
            print(e)

In [85]:
def search(query="", limit=10):
    before = time.time()
    vec = text2vec(query)
    vec_took = time.time() - before

    before = time.time()
    near_vec = {"vector": vec}
    res = client \
        .query.get("SQLQuestions", ["sql", "_additional {certainty}"]) \
        .with_near_vector(near_vec) \
        .with_limit(limit) \
        .do()
    search_took = time.time() - before

    # print("\nQuery \"{}\" with {} results took {:.3f}s ({:.3f}s to vectorize and {:.3f}s to search)" \
          # .format(query, limit, vec_took+search_took, vec_took, search_took))
    # print(res)
    for post in res["data"]["Get"]["SQLQuestions"]:
        # print("{:.4f}: {}".format(post["_additional"]["certainty"], post["question"]))
        print(post["sql"])
        # print('---')

In [79]:
vectors = vectorize_questions(questions)

Vectorized 429 items in 39.825576305389404s


In [80]:
import_questions_with_vectors(questions, sqls, vectors, client)

In [86]:
search("who has the highest budget in last month", 10)


SELECT u.Name, SUM(t.Budget) FROM Users AS u join Tasks AS t on u.Id = t.AssigneeId group by u.Name ORDER BY SUM(t.Budget) DESC limit 1;

SELECT u.Name, t.Name FROM Tasks AS t JOIN Users as u ON t.AssigneeId = u.Id WHERE t.Completed = 0 AND t.DueOn > CURRENT_DATE() ORDER BY t.Budget DESC LIMIT 1;

SELECT s.Name, SUM(t.Budget) AS budget FROM Sections AS s inner join Tasks AS t on s.Id = t.sectionId group by s.Name ORDER BY budget DESC limit 1;

SELECT u.Name, COUNT(*) AS COUNT_overdue FROM Users AS u INNER JOIN Tasks AS t ON u.Id = t.AssigneeId WHERE t.Completed = 0 and t.DueOn < now() and Priority LIKE '%Low%' and budget < 300 group by u.ID ORDER BY COUNT_overdue DESC;

SELECT name, budget FROM Tasks ORDER BY budget DESC limit 5;

SELECT AVG(budget), month(CompletedAt), year(CompletedAt) FROM Tasks AS t inner join Users AS u on u.Id = t.AssigneeId WHERE u.Name LIKE '%Shir%' group by month(CompletedAt), year(CompletedAt);

SELECT Tasks.Budget, Tasks.Name FROM Tasks WHERE Tasks.Budget = 

In [ ]:
#
